# Benchmarking

You can submit protocols for benchmarking using the `rex` workflow scripting language.

In [1]:
from rush import build_blocking_provider_with_functions

In [2]:
# |hide
# hidden setup for the notebook
import os
import pathlib

WORK_DIR = pathlib.Path("~/qdx/benchmark_notebook").expanduser()
if WORK_DIR.exists():
    !rm -r $WORK_DIR
os.makedirs(WORK_DIR, exist_ok=True)
# swap into clean workdir so that our tests are deterministic
os.chdir(WORK_DIR)
PUT_YOUR_TOKEN_HERE = os.environ["RUSH_TOKEN"]
PUT_YOUR_PREFERRED_WORKING_DIRECTORY_HERE = WORK_DIR
os.environ["RUSH_RESTORE_BY_DEFAULT"] = "False"

In [3]:
client = build_blocking_provider_with_functions(
    access_token=PUT_YOUR_TOKEN_HERE
    # for example, if your token is 00000000-dddd-cccc-0000-11111111,
    # then you should put access_token="00000000-dddd-cccc-0000-11111111"
    # (including the double quotes)
)

2025-01-30 17:41:51,234 - rush - INFO - Not restoring by default via env


In [4]:
benchmark = client.benchmark_blocking(name="OpenFF CDK2 Protein Ligand Benchmark")

In [5]:
# |hide
from IPython.display import Markdown as md
rex_code_above = """
let
    runspec = RunSpec {
        target = 'Bullet',
        resources = Resources {
            storage = some 10,
            storage_units = some "MB",
            gpus = some 1
        }
    },

    runspec_nogpu = RunSpec {
        target = 'Bullet',
        resources = Resources {
            storage = some 10,
            storage_units = some "MB",
            gpus = none
        }
    },

    auto3d = \\smi ->
        let
            result = get 0 (auto3d_rex_s runspec { k = 1 } [smi]),
            make_virtual_object = \\index ->
                VirtualObject {
                    path = get "path" (get index result),
                    size = get "size" (get index result),
                    format = "json"
                }
        in
            (make_virtual_object 0, make_virtual_object 1),

    p2rank = \\prot_conf ->  p2rank_rex_s runspec_nogpu {} prot_conf,

    gnina = \\prot_conf -> \\bounding_box -> \\smol_conf ->
        get 0 (get 0 (gnina_rex_s runspec {} [prot_conf] [bounding_box] smol_conf [])),

in
\\input ->
    let
        protein = load (id (get 0 (inputs input))) 'ProteinConformer',
        smol_id = id (get 1 (inputs input)),
        smiles = smi (load smol_id 'Smol'),

        structure = load (structure_id protein) 'Structure',
        trc = [
            topology structure,
            residues structure,
            chains structure
        ],

        bounding_box = get 0 (get 0 (p2rank trc)),

        smol_structure = auto3d smiles,

        docked_structure = gnina trc bounding_box [smol_structure],

        min_affinity = list_min (map (get "affinity") (get "scores" docked_structure)),

        binding_affinity = BindingAffinity {
            affinity = min_affinity,
            affinity_metric = 'kcal/mol',
            protein_id = protein_id protein,
            smol_id = smol_id,
            metadata = Metadata {
                name = id input,
                description = none,
                tags = [id input]
            }
        }
    in
        [BenchmarkArg {
            entity = "BindingAffinity",
            id = save binding_affinity
        }]
"""

In [6]:
md(f"```haskell{rex_code_above}```")

```haskell
let
    runspec = RunSpec {
        target = 'Bullet',
        resources = Resources {
            storage = some 10,
            storage_units = some "MB",
            gpus = some 1
        }
    },

    runspec_nogpu = RunSpec {
        target = 'Bullet',
        resources = Resources {
            storage = some 10,
            storage_units = some "MB",
            gpus = none
        }
    },

    auto3d = \smi ->
        let
            result = get 0 (auto3d_rex_s runspec { k = 1 } [smi]),
            make_virtual_object = \index ->
                VirtualObject {
                    path = get "path" (get index result),
                    size = get "size" (get index result),
                    format = "json"
                }
        in
            (make_virtual_object 0, make_virtual_object 1),

    p2rank = \prot_conf ->  p2rank_rex_s runspec_nogpu {} prot_conf,

    gnina = \prot_conf -> \bounding_box -> \smol_conf ->
        get 0 (get 0 (gnina_rex_s runspec {} [prot_conf] [bounding_box] smol_conf [])),

in
\input ->
    let
        protein = load (id (get 0 (inputs input))) 'ProteinConformer',
        smol_id = id (get 1 (inputs input)),
        smiles = smi (load smol_id 'Smol'),

        structure = load (structure_id protein) 'Structure',
        trc = [
            topology structure,
            residues structure,
            chains structure
        ],

        bounding_box = get 0 (get 0 (p2rank trc)),

        smol_structure = auto3d smiles,

        docked_structure = gnina trc bounding_box [smol_structure],

        min_affinity = list_min (map (get "affinity") (get "scores" docked_structure)),

        binding_affinity = BindingAffinity {
            affinity = min_affinity,
            affinity_metric = 'kcal/mol',
            protein_id = protein_id protein,
            smol_id = smol_id,
            metadata = Metadata {
                name = id input,
                description = none,
                tags = [id input]
            }
        }
    in
        [BenchmarkArg {
            entity = "BindingAffinity",
            id = save binding_affinity
        }]
```

In [7]:
submission = client.run_benchmark_blocking(benchmark.id, rex_code_above, "simple submission", sample=0.2)

View your submission at https://rush-qdx-2-staging.web.app/projects/e67f6166-5c1f-4305-ac30-d62a1c9b2d31/runs?selectedRunId=1f31217b-a338-4357-aa5f-baae5084c451
